In [1]:
# 「Pythonによる因果分析」から抜粋
import numpy as np
import pandas as pd

In [2]:
num_data = 10
x1 = np.random.choice([0,1], num_data, p=[0.6,0.4])

x2 = np.random.choice([0,1], num_data, p=[0.4,0.6])

df = pd.DataFrame({
    'x1':x1,
    'x2':x2,
    })
df.head()

,x1,x2
0,0,1
1,0,1
2,1,0
3,0,1
4,1,0


In [3]:
x3 = []
for i in range(num_data):
    if x1[i] == 0 and x2[i] == 0:
        x3_value = np.random.choice([0,1], 1 , p=[0.2, 0.8])
        x3.append(x3_value[0])
    elif x1[i] == 0 and x2[i] == 1:
        x3_value = np.random.choice([0,1], 1 , p=[0.3, 0.7])
        x3.append(x3_value[0])
    elif x1[i] == 1 and x2[i] == 0:
        x3_value = np.random.choice([0,1], 1 , p=[0.4, 0.6])
        x3.append(x3_value[0])
    elif x1[i] == 1 and x2[i] == 1:
        x3_value = np.random.choice([0,1], 1 , p=[0.1, 0.9])
        x3.append(x3_value[0])
        
df["x3"] = x3
df

,x1,x2,x3
0,0,1,1
1,0,1,1
2,1,0,0
3,0,1,1
4,1,0,1
5,0,1,1
6,0,0,1
7,1,1,1
8,1,0,0
9,1,0,1


In [4]:
from pgmpy.models import BayesianModel
model = BayesianModel([('x1','x3'),('x2','x3')])

In [5]:
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, df)
print("\n", pe.state_counts('x1'))
print("\n", pe.state_counts('x2'))
print("\n", pe.state_counts('x3'))


    x1
0   5
1   5

    x2
0   5
1   5

 x1    0         1     
x2    0    1    0    1
x3                    
0   0.0  0.0  2.0  0.0
1   1.0  4.0  2.0  1.0


In [6]:
from pgmpy.estimators import BayesianEstimator

estimator = BayesianEstimator(model, df)

cpd_x1 = estimator.estimate_cpd(
        'x1', prior_type="dirichlet", pseudo_counts=[[0],[0]])
cpd_x2 = estimator.estimate_cpd(
        'x2', prior_type="dirichlet", pseudo_counts=[[0],[0]])
cpd_x3 = estimator.estimate_cpd(
        'x3', prior_type="dirichlet", pseudo_counts=[[0,0,0,0],[0,0,0,0]])

print(cpd_x1)
print(cpd_x2)
print(cpd_x3)

+-------+-----+
| x1(0) | 0.5 |
+-------+-----+
| x1(1) | 0.5 |
+-------+-----+
+-------+-----+
| x2(0) | 0.5 |
+-------+-----+
| x2(1) | 0.5 |
+-------+-----+
+-------+-------+-------+-------+-------+
| x1    | x1(0) | x1(0) | x1(1) | x1(1) |
+-------+-------+-------+-------+-------+
| x2    | x2(0) | x2(1) | x2(0) | x2(1) |
+-------+-------+-------+-------+-------+
| x3(0) | 0.0   | 0.0   | 0.5   | 0.0   |
+-------+-------+-------+-------+-------+
| x3(1) | 1.0   | 1.0   | 0.5   | 1.0   |
+-------+-------+-------+-------+-------+


In [7]:
from pgmpy.estimators import BicScore
bic = BicScore(df)
print(bic.score(model))

-23.543287612420833


In [8]:
from pgmpy.models import BayesianModel
model = BayesianModel([('x2','x1'),('x2','x3')])
bic = BicScore(df)
print(bic.score(model))

-21.05701710851273
